In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
data = pd.read_csv("../../../Data/Dataset_splited.csv")

### Get features categorie

In [3]:
def categorisation(data,date_cols,target="price"):
    distance = []
    date = []
    numerique = []
    categorique = []
    for col in data.columns:
        if data[col].dtypes == np.object:
            try:
                if str(type(eval(data[col][data[col].first_valid_index()]))) == "<class 'list'>":
                    distance.append(col)
                elif col in date_cols:
                    date.append(col)
                else:
                    categorique.append(col)
            except:
                if col in date_cols:
                    date.append(col)
                else:
                    categorique.append(col)
                
            
        else :
            if col != target:
                numerique.append(col)
                
    return dict({"numerique":numerique,"categorique":categorique,"date":date,"distance":distance})

feat_cat = categorisation(data,['listing_date'])

In [4]:
feat_cat

{'numerique': ['bathrooms',
  'bedrooms',
  'erf_size',
  'floor_size',
  'garages',
  'listing_number',
  'pet_friendly',
  'garden',
  'pool',
  'secure_parking',
  'parking',
  'no_pets_allowed',
  'furnished',
  'flatlet',
  'fibre_internet',
  'furnished_(optional)',
  'adsl_internet',
  'satellite_internet',
  'fixed_wimax_internet',
  'split'],
 'categorique': ['town', 'type_of_property'],
 'date': ['listing_date'],
 'distance': []}

### Fill by 0 those features (we suppose missing == 0) except 'bathrooms' and 'bedrooms'

In [5]:
_feat = [
  'garages',
  'pet_friendly',
  'garden',
  'pool',
  'secure_parking',
  'parking',
  'no_pets_allowed',
  'furnished',
  'flatlet',
  'fibre_internet',
  'furnished_(optional)',
  'adsl_internet',
  'satellite_internet',
  'fixed_wimax_internet']

In [6]:
for c in _feat:
    print(data[c].unique())
    data[c].fillna(0,inplace=True)

[ nan  1.   2.   3.   4.   6.   7.   5.  10.  20.   8.   1.5]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  2.  1.  6.  4.  8.  3. 10. 15.  5. 12.  9. 11.]
[nan  1.  3.  2.  6.  5.  4. 30.  7.  8. 10.  9. 11. 13. 12. 50.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]
[nan  1.]


In [7]:
for c in ['bathrooms','bedrooms']:
    print(data[c].unique())
    data[c].fillna(1,inplace=True)

[ 1.   2.   2.5  5.   4.5  1.5  7.5  3.   4.   3.5 12.   5.5  nan  7.
  6.5  6.   0.5 30.   9.  18.  11.  15.   8.   9.5  8.5 10.  25.  13.
 14. ]
[ 2.   nan  3.   4.   1.   6.   7.   8.   5.  12.   0.5  1.5  9.  30.
 11.  16.  15.  10.   2.5  3.5 18.   4.5 13.  33.  14. ]


### the rest fill by median

In [8]:
rest_of_num = ['erf_size','floor_size']

In [9]:
data[rest_of_num] = data[rest_of_num].fillna(data[rest_of_num].median())

### Listing date preprocessing
Get year

In [10]:
data['listing_date'] = [v.year for v in pd.to_datetime(data['listing_date'])]

In [11]:
data['listing_date'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015])

### target mean ordering encoding of categorical value "type_of_property" based on train dataset

In [12]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'price']].groupby(feature).mean()['price']
    #print(frame[[feature, 'a']].groupby(feature).mean()['a'])
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    #print(ordering)
    ordering = ordering['ordering'].to_dict()
    #print("Encoding results : ",ordering)
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
        
    return ordering

#tmp = pd.DataFrame({"a":[1,2,5,10],"b":["1","1","o","o"]})

qual_encoded = []
train = data[data["split"]==0].copy()
for q in ['type_of_property']:  
    qual_encoded.append(encode(train, q))

We can see it follow the distribution order from boxplot in advanced EDA

#### Then encode the entire dataset

In [13]:
data["type_of_property"] = data["type_of_property"].replace(qual_encoded[0])


In [14]:
data["type_of_property"].unique()

array([1, 2, 3])

### Town feature preprocess -> get only the district (quartier)

In [15]:
series = []
for v in data["town"]:
    v = v.split(',')
    v.pop()
    series.append(v.pop())
    
data["town"] = series

target mean ordering encoding of categorical value "town" based on all dataset

In [16]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'price']].groupby(feature).mean()['price']
    #print(frame[[feature, 'a']].groupby(feature).mean()['a'])
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    #print(ordering)
    ordering = ordering['ordering'].to_dict()
    #print("Encoding results : ",ordering)
    
    #for cat, o in ordering.items():
        #frame.loc[frame[feature] == cat, feature+'_E'] = o
        
    return ordering

#tmp = pd.DataFrame({"a":[1,2,5,10],"b":["1","1","o","o"]})

qual_encoded = []
for q in ['town']:  
    qual_encoded.append(encode(data, q))

In [17]:
data["town"] = data["town"].replace(qual_encoded[0])


In [18]:
data["town"].unique()

array([178, 236, 142, 152,  56, 246, 134, 179, 171, 150,  37, 186, 222,
       176, 197, 210, 243, 164, 191, 192, 105, 162, 235, 183,  99, 204,
       181,  67, 112, 175, 126,  65, 185, 102, 207, 166,  81, 147,  86,
        80, 221,  45, 199, 124, 116, 120, 198,  83, 115, 203, 193, 111,
        34, 177,  71, 114, 145, 117, 189, 122, 211,  76, 184, 234,  49,
       242, 223, 224,  10, 249,  14,  82,  31, 100, 214, 169, 188, 195,
        70, 156, 133, 228,  21,  79, 201, 101, 244, 241, 196, 160, 200,
        98, 213, 158,  66,  91, 130,  50,  38, 194, 216, 232, 208, 155,
       225, 174, 247, 238,   3, 180, 231, 227,  62,  20,  85, 136, 190,
       146, 151, 128,  78,  95, 205, 170, 106, 245, 159,  92,  43,  29,
       135,  17,  94,  48,  12, 113, 209, 240, 123, 143,  54, 144,  15,
       104, 230,  53, 239,  41, 157,  63,  47, 154,  75, 149,  96,  13,
       173,  59, 220,   2,  58,  55, 182, 163, 165, 132, 118, 141, 148,
       119,  57,  19, 217,  40, 237, 110,  26,  72, 215,   8,  5

#### drop listing_number 'cause just index value of property 

In [19]:
data = data.drop(['listing_number'],axis=1)

### scale continous/discrete numeric features 

In [20]:
to_scale = ['bathrooms',  'bedrooms',  'erf_size',  'floor_size',  'garages',  'secure_parking',  'parking'] 

# Get mean and deviation of train
#mean = data[data["split"]==0][to_scale].mean()
#std = data[data["split"]==0][to_scale].std()

#data[to_scale] = data[to_scale]-mean
#data[to_scale] = data[to_scale]/std

#log transform
data[to_scale]=np.log(data[to_scale])
data[to_scale]=data[to_scale].replace(-np.inf,-1)
data[to_scale]

,bathrooms,bedrooms,erf_size,floor_size,garages,secure_parking,parking
0,0.000000,0.693147,5.894403,4.430817,-1.000000,-1.0,-1.0
1,0.693147,0.693147,5.894403,4.574711,-1.000000,-1.0,-1.0
2,0.000000,0.693147,5.894403,4.007333,-1.000000,-1.0,0.0
3,0.000000,0.000000,5.894403,3.637586,-1.000000,-1.0,-1.0
4,0.000000,0.693147,5.298317,4.430817,0.000000,-1.0,0.0
...,...,...,...,...,...,...,...
7841,0.693147,1.098612,5.894403,4.812184,0.693147,-1.0,-1.0
7842,0.000000,0.693147,8.727940,4.007333,-1.000000,-1.0,0.0
7843,0.000000,0.000000,5.894403,4.430817,-1.000000,-1.0,-1.0
7844,1.386294,1.609438,7.600902,5.942799,0.693147,-1.0,-1.0


In [21]:
data.isna().sum()

bathrooms               0
bedrooms                0
erf_size                0
floor_size              0
garages                 0
listing_date            0
pet_friendly            0
price                   0
town                    0
type_of_property        0
garden                  0
pool                    0
secure_parking          0
parking                 0
no_pets_allowed         0
furnished               0
flatlet                 0
fibre_internet          0
furnished_(optional)    0
adsl_internet           0
satellite_internet      0
fixed_wimax_internet    0
split                   0
dtype: int64

### data saving

In [22]:
data.to_csv("../../../Data/Dataset_splited_v1.csv",index=False)